<a href="https://colab.research.google.com/github/alinelena/janus-lavello/blob/main/hello_janus.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

install dependencies, pay special atention to install ase by hand

In [ ]:
!python3 -m pip install -U git+https://github.com/stfc/janus-core.git
!python3 -m pip install -U git+https://gitlab.com/ase/ase.git
!python3 -m pip install rdkit

In [ ]:
from ase.build import bulk
from ase.io import write

from janus_core.single_point import SinglePoint

NaCl = bulk('NaCl', 'rocksalt', a=5.63, cubic=True)
sp = SinglePoint(struct=NaCl,
                 architecture="mace_mp",
                 device='cpu',
                 calc_kwargs={'model':'medium'})

res = sp.run()
print(res)

In [ ]:
from ase.build import molecule
from ase.io import write

from janus_core.single_point import SinglePoint

sp = SinglePoint(struct=molecule('H2O'),
                 architecture="mace_off",
                 device='cpu',
                 calc_kwargs={'model':'medium'})

res = sp.run()
print(res)

In [ ]:
from ase.build import surface,bulk, add_adsorbate, molecule
from ase.visualize import view
from ase.io import read
from rdkit import Chem
m = Chem.MolFromSmiles('C(C1C(C(C(C(O1)OC2(C(C(C(O2)CO)O)O)CO)O)O)O)O')
Chem.SDWriter('sugar.sdf').write(m)

a = 5.63
NaCl = bulk('NaCl', crystalstructure="rocksalt",cubic=True,orthorhombic=True,a=5.63)*(6,6,3)
NaCl.center(vacuum=20.0, axis=2)
sugar = read("sugar.sdf")
add_adsorbate(slab=NaCl, adsorbate=sugar, height=4.5, position=(10,10))
write("slab.xyz",NaCl)
sp = SinglePoint(struct_path="slab.xyz",
                 architecture="mace_mp",
                 device='cpu',
                 calc_kwargs={'model':'small'})
res = sp.run()
print(res)
view(NaCl,viewer='x3d')


In [ ]:

print(m)

In [ ]:
from janus_core.geom_opt import optimize
from janus_core.mlip_calculators import choose_calculator
from ase.filters import FrechetCellFilter

NaCl = bulk('NaCl', 'rocksalt', a=5.62, cubic=True)
NaCl.set_cell([5.62,5.62,5.62,89,90,90])
nacl_one = NaCl.copy()
nacl_one.calc = choose_calculator(architecture="mace_mp", model="small")

opt_NaCl = optimize(nacl_one)
print(opt_NaCl.cell)
print(opt_NaCl.cell.angles())

nacl_two = NaCl.copy()
nacl_two.calc = choose_calculator(architecture="mace_mp", model="small")

opt_NaCl = optimize(nacl_two,fmax=0.0001,filter_func=FrechetCellFilter,dyn_kwargs={'steps':10},filter_kwargs={'hydrostatic_strain':False})
print(opt_NaCl.cell)
print(opt_NaCl.cell.angles())

nacl_three = NaCl.copy()
nacl_three.calc = choose_calculator(architecture="mace_mp", model="small")
opt_NaCl = optimize(nacl_three,filter_func=FrechetCellFilter,filter_kwargs={'hydrostatic_strain':True})
print(opt_NaCl.cell)
print(opt_NaCl.cell.angles())